In [31]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00


**<h4> Business Objective:**

**Objective:**The primary goal of this notebook is to enhance the search engine for an ecommerce platform. By leveraging advanced natural language processing (NLP) techniques to significantly improve the relevance of search results.

**<h4> Loading DataSet:**

In [32]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import openai

In [ ]:
#clone the git repo that contains the data and additional information about the dataset
!git clone https://github.com/wayfair/WANDS.git

Cloning into 'WANDS'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 40 (delta 7), reused 23 (delta 3), pack-reused 0
Receiving objects: 100% (40/40), 33.32 MiB | 14.00 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Updating files: 100% (19/19), done.


In [ ]:
# get search queries
query_df = pd.read_csv("WANDS/dataset/query.csv", sep='\t')
query_df.head()

,query_id,query,query_class
0,0,salon chair,Massage Chairs
1,1,smart coffee table,Coffee & Cocktail Tables
2,2,dinosaur,Kids Wall Décor
3,3,turquoise pillows,Accent Pillows
4,4,chair and a half recliner,Recliners


In [16]:
query_df.head(100).to_csv('/content/drive/MyDrive/Harvard/query.csv')

In [ ]:
# get products
product_df = pd.read_csv("WANDS/dataset/product.csv", sep='\t')
product_df.head()

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0


In [17]:
product_df.head(100).to_csv('/content/drive/MyDrive/Harvard/product.csv')

In [ ]:
# get manually labeled groundtruth lables
label_df = pd.read_csv("WANDS/dataset/label.csv", sep='\t')
label_df.head()

,id,query_id,product_id,label
0,0,0,25434,Exact
1,1,0,12088,Irrelevant
2,2,0,42931,Exact
3,3,0,2636,Exact
4,4,0,42923,Exact


In [18]:
label_df.head(100).to_csv('/content/drive/MyDrive/Harvard/lable.csv')

**<h4> Data Visualization:**

* Considering the time doing very basic visualization.

In [19]:
print(product_df.shape)
print(label_df.shape)
print(query_df.shape)

(42994, 9)
(233448, 4)
(480, 3)


In [21]:
product_df.isnull().sum()

product_id                0
product_name              0
product_class          2852
category hierarchy     1556
product_description    6008
product_features          0
rating_count           9452
average_rating         9452
review_count           9452
dtype: int64

In [22]:
query_df.isnull().sum()

query_id       0
query          0
query_class    6
dtype: int64

In [23]:
label_df.isnull().sum()

id            0
query_id      0
product_id    0
label         0
dtype: int64

Note: We have nulls in the dataset we need to handle them in preprocessing.

**<h4> Data Preprocessing:**

* **Handling Null Values:** addressing potential missing data in key text columns—product_name, category hierarchy, and product_description—by filling any null values with an empty string ('').

* **Product Names and Category Hierarchy:** We apply a simple lowercase transformation to standardize the text, which helps in maintaining consistency and reducing complexity during the search process.

* **Product Descriptions:** This column undergoes a more extensive preprocessing Converting to lowercase, removing stopwords,lemmatization to enhance the meaningful content description.

**Note:** 1) If we are using LLM models we can skip this step as they can understand context without any preprocessing.

2) I see that adding Category Hierarchy increases MAE by 10% so adding new column for the given code.

In [24]:
# Handling nulls with empty string
product_df['product_name'] = product_df['product_name'].fillna('')
product_df['category hierarchy'] = product_df['category hierarchy'].fillna('')
product_df['product_description'] = product_df['product_description'].fillna('')

In [25]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

# Setup lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_description(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

# Lowercasing product names and categories
product_df['product_name'] = product_df['product_name'].str.lower()
product_df['category hierarchy'] = product_df['category hierarchy'].str.lower()

# Apply preprocessing to product descriptions
product_df['product_description'] = product_df['product_description'].apply(preprocess_description)

# Combine the columns and clean extra spaces
product_df['combined_text'] = product_df['product_name'] + " " + product_df['category hierarchy'] + " " + product_df['product_description']
product_df['combined_text'] = product_df['combined_text'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


**<h4> Modeling Embeding Creation:**

* Changing TF-IDF embeddings to OpenAI as it can capture the deeper semantic context of text, which is crucial for understanding the intent behind user queries.

* Implimented batching approach considering OpenAI context window limitage.

**Note:** To avoid the complexity i removed given tf-idf embeding logic.

In [34]:
# Set your OpenAI API key here
openai.api_key = ' '

In [35]:
def calculate_embeddings(dataframe, batch_size=70):
    """
    Calculate embeddings for combined product name and description using OpenAI's model,
    processing the data in batches to avoid exceeding API limits.

    Parameters:
    dataframe (pd.DataFrame): DataFrame with product_id, product_name, product_description, and other product information.
    batch_size (int): Number of records to process in each batch.

    Returns:
    np.array: Array of embeddings for the products.
    """
    # Combine product name and description
    combined_text = dataframe['combined_text']
    # Initialize an empty list to store all embeddings
    all_embeddings = []

    # Process each batch
    for i in range(0, len(combined_text), batch_size):
        batch_texts = combined_text[i:i + batch_size].tolist()
        embeddings = get_openai_embeddings(batch_texts)
        if embeddings is not None:
            all_embeddings.append(embeddings)

    # Concatenate all embeddings arrays together
    return np.concatenate(all_embeddings, axis=0)

def get_openai_embeddings(texts):
    """
    Get embeddings for a list of texts using OpenAI's API, ensuring proper JSON formatting.
    """
    try:
        response = openai.Embedding.create(
            input=texts,
            model="text-embedding-ada-002"
        )
        # Extract embeddings and convert to a numpy array
        embeddings = [data['embedding'] for data in response['data']]
        return np.array(embeddings)
    except Exception as e:
        print("Failed to fetch embeddings:", str(e))
        return None


**<h4> Similarity Search: :**

* Using embeddings implementing a search function that computes cosine similarities between query embeddings and product embeddings to retrieve the top products.

In [36]:
product_embeddings=calculate_embeddings(product_df)

In [37]:
def get_top_product_ids_for_query(product_embeddings, query, top_n=10):
    """
    Get top N products for a given query based on embedding similarity.

    Parameters:
    product_embeddings (np.array): Precomputed embeddings for products.
    query (str): Search query.
    top_n (int): Number of top products to return.

    Returns:
    np.array: Indices of the top N product embeddings most similar to the query.
    """
    # Convert query to embedding
    query_embedding = get_openai_embeddings([query])[0]  # Assume single query
    #print(query_embedding)
    # Calculate cosine similarities
    cosine_similarities = cosine_similarity([query_embedding], product_embeddings).flatten()
    # Find top N products
    top_product_indices = cosine_similarities.argsort()[-top_n:][::-1]
    return top_product_indices

#define the test query
query = "armchair"

#obtain top product IDs
top_product_ids = get_top_product_ids_for_query(product_embeddings,query)

print(f"Top products for '{query}':")
for product_id in top_product_ids:
    product = product_df.loc[product_df['product_id'] == product_id]
    print(product_id, product['product_name'].values[0])

Top products for 'armchair':
29626 armchair
25745 johannes 29.1 '' wide armchair
28850 callie armchair
36059 arm chair in blue
26978 orchard hill 20.47 '' wide armchair
19159 dandir 50 '' wide armchair
37322 28.5 '' wide polyester armchair
42392 killingly armchair
1230 faux leather armchair
10777 crutchfield 33.5 '' wide top grain leather armchair


**<h4> Evaluation:**

In [38]:
#define functions for evaluating retrieval performance
def map_at_k(true_ids, predicted_ids, k=10):
    """
    Calculate the Mean Average Precision at K (MAP@K).

    Parameters:
    true_ids (list): List of relevant product IDs.
    predicted_ids (list): List of predicted product IDs.
    k (int): Number of top elements to consider.
             NOTE: IF you wish to change top k, please provide a justification for choosing the new value

    Returns:
    float: MAP@K score.
    """
    #if either list is empty, return 0
    if not len(true_ids) or not len(predicted_ids):
        return 0.0

    score = 0.0
    num_hits = 0.0

    for i, p_id in enumerate(predicted_ids[:k]):
        if p_id in true_ids and p_id not in predicted_ids[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    return score / min(len(true_ids), k)

In [39]:
grouped_label_df = label_df.groupby('query_id')

In [40]:
#implementing a function to retrieve exact match product IDs for a query_id
def get_exact_matches_for_query(query_id):
    query_group = grouped_label_df.get_group(query_id)
    exact_matches = query_group.loc[query_group['label'] == 'Exact']['product_id'].values
    return exact_matches

#applying the function to obtain top product IDs and adding top K product IDs to the dataframe
#query_df['top_product_ids'] = query_df['query'].apply(get_top_product_ids_for_query)
query_df['top_product_ids'] = query_df['query'].apply(lambda query: get_top_product_ids_for_query(product_embeddings, query))


#adding the list of exact match product_IDs from labels_df
query_df['relevant_ids'] = query_df['query_id'].apply(get_exact_matches_for_query)

#now assign the map@k score
query_df['map@k'] = query_df.apply(lambda x: map_at_k(x['relevant_ids'], x['top_product_ids'], k=10), axis=1)

In [41]:
# calculate the MAP across the entire query set
query_df.loc[:, 'map@k'].mean()

0.4042485164976484

**New Evaluation Function Weighted Mean Average Precision (Weighted MAP):**

* This function helps to address the issue of partial matches being treated as irrelevant.
* Similar to the standard MAP, but each hit (match) is weighted by its relevance score. If a product is a perfect match, it receives a weight of 1 and for partial matches receive weights according to their relevance scores.

* Assign a fractional score to partial matches based on the degree of relevance. For instance, a 50% keyword overlap with the query might score 0.5, while a 25% overlap might score 0.25.

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_partial_match_scores(query, products, threshold=0.1):
    """
    Calculate partial match scores based on keyword overlap using cosine similarity.

    Parameters:
    query (str): The search query.
    products (DataFrame): DataFrame containing product descriptions and IDs.
    threshold (float): Minimum similarity score to consider as a partial match.

    Returns:
    dict: Dictionary of product IDs and their partial match scores.
    """
    # Vectorize the query and the product descriptions
    vectorizer = TfidfVectorizer()
    combined_text = products['product_description'].tolist() + [query]
    tfidf_matrix = vectorizer.fit_transform(combined_text)

    # Calculate cosine similarities between the query and all products
    cosine_sim = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()

    # Filter products that meet the similarity threshold and assign scores
    partial_matches = {products.iloc[i]['product_id']: cosine_sim[i] for i in range(len(cosine_sim)) if cosine_sim[i] > threshold}

    return partial_matches

# Usage
partial_matches = get_partial_match_scores(query, product_df)


In [61]:
def weighted_map_at_k(true_ids, predicted_ids, k=10, partial_matches=None):
    """
    Calculate the weighted Mean Average Precision at K (MAP@K) that accounts for partial matches.

    Parameters:
    true_ids (list): List of IDs considered as perfect matches.
    predicted_ids (list): List of predicted product IDs from the search engine.
    k (int): Number of top elements to consider.
    partial_matches (dict): A dictionary mapping product IDs to a score reflecting the degree of partial match.

    Returns:
    float: Adjusted MAP@K score.
    """
    if not len(true_ids) or not len(predicted_ids):
        return 0.0

    score = 0.0
    num_hits = 0.0

    for i, p_id in enumerate(predicted_ids[:k]):
        if p_id in true_ids:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
        elif p_id in partial_matches:
            # Add partial match score proportionally to its rank
            num_hits += partial_matches[p_id]
            score += num_hits / (i + 1.0)

    return score / min(len(true_ids), k)



In [65]:
#implementing a function to retrieve exact match product IDs for a query_id
def get_exact_matches_for_query(query_id):
    query_group = grouped_label_df.get_group(query_id)
    exact_matches = query_group.loc[query_group['label'] == 'Exact']['product_id'].values
    return exact_matches

#applying the function to obtain top product IDs and adding top K product IDs to the dataframe
#query_df['top_product_ids'] = query_df['query'].apply(get_top_product_ids_for_query)
query_df['top_product_ids'] = query_df['query'].apply(lambda query: get_top_product_ids_for_query(product_embeddings, query))


#adding the list of exact match product_IDs from labels_df
query_df['relevant_ids'] = query_df['query_id'].apply(get_exact_matches_for_query)

#now assign the map@k score
#query_df['map@k'] = query_df.apply(lambda x: adjusted_map_at_k(x['relevant_ids'], x['top_product_ids'], k=10), axis=1)

# When applying the function, ensure partial_matches is passed correctly
query_df['map@k'] = query_df.apply(lambda x: weighted_map_at_k(x['relevant_ids'], x['top_product_ids'], k=10, partial_matches=partial_matches), axis=1)

In [66]:
# calculate the MAP across the entire query set
query_df.loc[:, 'map@k'].mean()

0.4072175704648924

Note: As we don't have significant partial matches there is a little increment in the MAP score.

**Trade-offs**

**Complexity:** Introducing weighted scores increases the complexity of the evaluation metric. Determining appropriate weights requires careful consideration and possibly additional data analysis.

**Subjectivity:** The determination of what constitutes a partial match and how much it should be scored can be somewhat subjective and may require tuning based on user feedback or empirical data.